## Messari standardised subgraphs

https://github.com/messari/subgraphs

### VVS Finance

In [7]:
import pandas as pd
import numpy as np
import time
import requests
import datetime as dt

In [8]:
endpoint = 'https://graph.cronoslabs.com/subgraphs/name/messari/vvs-finance'

In [9]:
#Check if subgraph is up to date
query = """{
  _meta{
    block{
  		hash
      number
    }
    hasIndexingErrors
    deployment
  }
}"""
request = requests.post(endpoint, json={'query':query})
data = request.json()
#Check if it is ok to run i.e. within 5 days of today
int(data['data']['_meta']['block'], base = 0) > (dt.datetime.today().timestamp() - 3600*24*5)

TypeError: int() can't convert non-string with explicit base

In [10]:
data

{'data': {'_meta': {'block': {'hash': '0x55e8102f2f3e687014963468c51f56b449b133be11819987571256872cfa79b9',
    'number': 4509736},
   'hasIndexingErrors': False,
   'deployment': 'QmeEKnjqXeSpwaqZy2ErhjA7L7v5rKnYdmJ3wgu4sRCTcc'}}}

6-Month Average Daily Volume

In [11]:
query = """{
  financialsDailySnapshots(first: 183, orderBy: timestamp, orderDirection: desc){
    id
    protocol{
      name
    }
    dailyVolumeUSD
    dailyTotalRevenueUSD
    dailySupplySideRevenueUSD
    dailyProtocolSideRevenueUSD
    timestamp
    blockNumber
  }
}
"""

In [12]:
request = requests.post(endpoint, json={'query':query})
data = request.json()

In [13]:
df = pd.json_normalize(data['data']['financialsDailySnapshots'])

In [14]:
df

,id,dailyVolumeUSD,dailyTotalRevenueUSD,dailySupplySideRevenueUSD,dailyProtocolSideRevenueUSD,timestamp,blockNumber,protocol.name
0,19241,1190768.886513239391107441556655335,3572.306659539718173322324669966004,2381.537773026478782214883113310666,1190.768886513239391107441556655335,1662481291,4509738,VVS Finance
1,19240,896483.0907708036684201159920118127,2689.449272312411005260347976035434,1792.966181541607336840231984023624,896.4830907708036684201159920118127,1662422380,4499329,VVS Finance
2,19239,1096139.484816084291839147503318073,3288.418454448252875517442509954215,2192.278969632168583678295006636145,1096.139484816084291839147503318073,1662335999,4484072,VVS Finance
3,19238,958334.571430100208091642801374814,2875.003714290300624274928404124444,1916.669142860200416183285602749632,958.334571430100208091642801374814,1662249588,4468802,VVS Finance
4,19237,1247014.193524339953047083414418144,3741.042580573019859141250243254433,2494.028387048679906094166828836296,1247.014193524339953047083414418144,1662163197,4453535,VVS Finance
...,...,...,...,...,...,...,...,...
178,19063,4828099.384884783015209280661115834,14484.29815465434904562784198334749,9656.198769769566030418561322231659,4828.099384884783015209280661115834,1647129580,1875181,VVS Finance
179,19062,5829387.959236297637541585573502189,17488.16387770889291262475672050658,11658.77591847259527508317114700438,5829.387959236297637541585573502189,1647043195,1860388,VVS Finance
180,19061,8104249.234263499836174721701987409,24312.74770279049950852416510596216,16208.49846852699967234944340397488,8104.249234263499836174721701987409,1646956798,1845630,VVS Finance
181,19060,5918802.29315851517819993242962691,17756.40687947554553459979728888068,11837.60458631703035639986485925378,5918.80229315851517819993242962691,1646870379,1830868,VVS Finance


In [15]:
df['dailyVolumeUSD'].astype(float).mean()

5676558.383805844

6-Month Average TVL

In [16]:
#DefiLlama https://defillama.com/protocols/dexes 
lma = pd.read_csv("https://api.llama.fi/dataset/vvs-finance.csv")

In [17]:
lma

,Unnamed: 0,Date,Timestamp,VVS Finance,VVS Finance.1,VVS Finance.2,VVS Finance.3,VVS Finance.4,VVS Finance.5,VVS Finance.6,...,VVS Finance.92,VVS Finance.93,VVS Finance.94,VVS Finance.95,VVS Finance.96,VVS Finance.97,VVS Finance.98,VVS Finance.99,VVS Finance.100,VVS Finance.101
0,NaN,NaN,NaN,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,...,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes
1,NaN,NaN,NaN,cronos,Total,cronos,cronos,cronos,cronos,cronos,...,Total,Total,Total,Total,Total,Total,Total,Total,Total,Total
2,NaN,NaN,NaN,TVL,TVL,Tokens(USD),Tokens(USD),Tokens(USD),Tokens(USD),Tokens(USD),...,Tokens,Tokens,Tokens,Tokens,Tokens,Tokens,Tokens,Tokens,Tokens,Tokens
3,NaN,NaN,NaN,NaN,NaN,crypto-com-chain,alethea-artificial-liquid-intelligence-token,SKY,DUSD,CROGE,...,CRONA,DARK,cosmos,CHIBIKI,DAI,dogecoin,NESS,TUSD,USDC,ADA
4,NaN,12/11/2021,1.636675e+09,289233660.20245445,289233660.20245445,289233660.20245445,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,NaN,02/09/2022,1.662077e+09,498796369.9597525,498796369.9597525,NaN,113223.92863096953,6037.618760421786,51234.00644970171,156.41519187528345,...,2498.4970151785556,8325746.763311804,142193.045991,1.70143485588353,1447.5289824267584,4077524.06215904,1584623.6501056065,4432009.260717055,174836306.683946,1354.265647
299,NaN,03/09/2022,1.662163e+09,497964561.22147703,497964561.22147703,NaN,114283.17720887427,6078.501054846286,51763.47403938493,153.56756634683663,...,2498.4970151785556,8153747.144127194,139042.786402,1.70143485588353,1419.8852444660138,4054126.510858,1618846.0301573942,4442724.772463327,174770900.972584,1378.220373
300,NaN,04/09/2022,1.662250e+09,495424367.5198908,495424367.5198908,NaN,113811.49447912403,6035.685681028244,50817.9015234918,149.1574081579085,...,2498.4970151785556,8149656.870214803,138775.264592,1.70143485588353,1399.4796741828577,3978269.41612028,1616018.1937171088,4442662.548842686,174648761.232706,1340.289314
301,NaN,05/09/2022,1.662336e+09,498291400.38260263,498291400.38260263,NaN,112177.29919872114,5947.779263957811,50815.20870483427,165.99470905216162,...,2498.4970151785556,8133224.556424989,134490.973336,1.70143485588353,1399.4796741828577,3995267.35527827,1505127.3933491255,4454243.224381304,174783799.779261,1309.559877


In [18]:
lma.iloc[:,3]

0                   Dexes
1                  cronos
2                     TVL
3                     NaN
4      289233660.20245445
              ...        
298     498796369.9597525
299    497964561.22147703
300     495424367.5198908
301    498291400.38260263
302      508372885.128933
Name: VVS Finance, Length: 303, dtype: object

In [19]:
#Just eth currently can take total by changing the header
lma.iloc[-183:, 3].astype(float).mean()

895864944.8120493

Largest Pool Concentration

In [20]:
df = pd.DataFrame()
ts = 0

while True:
    sl = len(df)
    query = """
{
  liquidityPools(first:1000, orderBy: createdTimestamp, orderDirection: asc, where: {createdTimestamp_gt: """+f'"{ts}"'+"""}){
    id
    name
    totalValueLockedUSD
    cumulativeVolumeUSD
    createdTimestamp
    createdBlockNumber
  }
}
"""
    request = requests.post(endpoint, json={'query':query})
    data = request.json()

    pool = pd.json_normalize(data['data']['liquidityPools'])
    df = df.append(pool)
    df.drop_duplicates(inplace=True)
    if len(df) == sl:
      break
    ts = df['createdTimestamp'].max()


/tmp/ipykernel_117953/2291472997.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pool)


In [21]:
df

,id,name,totalValueLockedUSD,cumulativeVolumeUSD,createdTimestamp,createdBlockNumber
0,0xe61db569e231b3f5530168aa2c9d50246525b6d6,VVS Finance Wrapped CRO/USD Coin,7276291.924057,1050573744.748573,1636431821,17798
1,0xa111c17f8b8303280d3eb01bbcd61000aa7f39f9,VVS Finance Wrapped CRO/Wrapped Ether,8599831.087633802160682308403341482,570882124.501524911498070361961484,1636431947,17821
2,0xbf62c67ea509e86f07c8c69d0286c0636c50270b,VVS Finance VVSToken/Wrapped CRO,50569188.93289926346561264512214109,0,1636431959,17823
3,0x8f09fff247b8fdb80461e5cf5e82dd1ae2ebd6d7,VVS Finance Wrapped BTC/Wrapped CRO,0,0,1636431969,17825
4,0xfd0cd0c651569d1e2e3c768ac0ffdab3c8f4844f,VVS Finance USD Coin/Wrapped Ether,67696.13325,1836952.393780991399387256042960667,1636431975,17826
...,...,...,...,...,...,...
903,0x6f9e2886bc263d9c2f0b795243edde6c250e1ba2,VVS Finance FerroToken/Fira,3.747191447475809746350295498865491,0,1662180008,4456506
904,0x4900c9e854fdc1db0b9dceab572319c29a41c924,VVS Finance Panterra Opal/HoloToken,2.188086269054424437322230801089404,0,1662350006,4486548
905,0x65c69d642cec2b71bbe0304278b496bfa183f235,VVS Finance WOO Network/HOD,0,0,1662350243,4486590
906,0x9aae44562397aecc8fdc83cac48f6b61244bdbcf,VVS Finance ATOM/ChainLink Token,0,0,1662350339,4486607


In [22]:
df['totalValueLockedUSD'].astype(float).max()/df['totalValueLockedUSD'].astype(float).sum()

0.7720430891620029

In [23]:
df['totalValueLockedUSD'].astype(float).nlargest(10).sum()/df['totalValueLockedUSD'].astype(float).sum()

0.9854267638135864

Liquidity Pools with > $1mn TVL

In [24]:
len(df.loc[df['totalValueLockedUSD'].astype(float) > 1000000])

12

Monthly Active Users

In [26]:
query = """{
  usageMetricsDailySnapshots(first: 183, orderBy: timestamp, orderDirection: desc){
    timestamp
    blockNumber
    id
    dailyActiveUsers
    dailySwapCount
    dailyDepositCount
    dailyWithdrawCount 
  }
}
"""

In [27]:
request = requests.post(endpoint, json={'query':query})
data = request.json()

In [28]:
df = pd.json_normalize(data['data']['usageMetricsDailySnapshots'])

In [30]:
df['dailyActiveUsers'].astype(float).mean()

3954.9180327868853

Implied slippage

In [31]:
query = """
{
  liquidityPools(first:10, orderBy: totalValueLockedUSD, orderDirection: desc){
    id
    name
    totalValueLockedUSD
    cumulativeVolumeUSD
    createdTimestamp
    createdBlockNumber
    inputTokens{
      id
      name
      symbol
      lastPriceUSD
      decimals
    }
    inputTokenBalances
  }
}
"""
request = requests.post(endpoint, json={'query':query})
if request.ok:
    data = request.json()

In [36]:
data['data']['liquidityPools'][3]

{'id': '0xa111c17f8b8303280d3eb01bbcd61000aa7f39f9',
 'name': 'VVS Finance Wrapped CRO/Wrapped Ether',
 'totalValueLockedUSD': '8599831.087633802160682308403341482',
 'cumulativeVolumeUSD': '570882124.501524911498070361961484',
 'createdTimestamp': '1636431947',
 'createdBlockNumber': '17821',
 'inputTokens': [{'id': '0x5c7f8a570d578ed84e63fdfa7b1ee72deae1ae23',
   'name': 'Wrapped CRO',
   'symbol': 'WCRO',
   'lastPriceUSD': '0',
   'decimals': 18},
  {'id': '0xe44fd7fcb2b1581822d0c862b68222998a0c299a',
   'name': 'Wrapped Ether',
   'symbol': 'WETH',
   'lastPriceUSD': '1659.847735364287680749184649674773',
   'decimals': 18}],
 'inputTokenBalances': ['72561229893458450359982900',
  '5159165945004724607551']}

In [37]:
slip1000 = []
slip10000 = []
slip100000 = []

for pool in data['data']['liquidityPools']:
    token0 = pool['inputTokens'][0]
    token0_reserveUSD = (float(pool['inputTokenBalances'][0])/10**token0['decimals'])*float(token0['lastPriceUSD'])
    
    token1 = pool['inputTokens'][1]
    token1_reserveUSD = (float(pool['inputTokenBalances'][1])/10**token1['decimals'])*float(token1['lastPriceUSD'])

    try:
        pi1000 = 1000/token0_reserveUSD
    except:
        pi1000 = 1000/token1_reserveUSD
    
    try:
        pi10000 = 10000/token0_reserveUSD
    except:
        pi10000 = 10000/token1_reserveUSD
    
    try:
        pi100000 = 100000/token0_reserveUSD
    except:
        pi100000 = 100000/token1_reserveUSD
    
    slip1000.append(pi1000)
    slip10000.append(pi10000)
    slip100000.append(pi100000)



In [38]:
slip1000

[6.349899816724522e-06,
 1.9795453491298922e-05,
 0.00022545502958664612,
 0.00011677563902411123,
 0.0001375066963896912,
 0.0007489763070299202,
 0.0007622187367320983,
 0.0007880807231733858,
 0.0009744867898414105,
 0.0005482439081910527]

In [39]:
np.mean(slip1000)

0.000432788918327634

In [40]:
np.mean(slip10000)

0.004327889183276339

In [41]:
np.mean(slip100000)

0.0432788918327634

In [ ]:
#Check price impact calculation is correct d0/r0